---
title: NFL Play-by-play Modeling and Classification
jupyter: python3
---

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sb
import statsmodels.api as sm
import sklearn.tree as tr
import sklearn.ensemble as ens
import sklearn.linear_model as slm
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

In [ ]:
nfl = pd.read_csv("NFL_play_by_play_2022.csv.gz")
nfl.shape

These data record play-by-play information for all games in the 2022 National Football League (NFL) season. These data were downloaded using the `nflverse` package for the R programming language (another statistics and data science environment), lightly edited, and saved in a tabular format for us to use in Python.

There are many measurements for each play, some of which are computed values from `nflverse`. Here's a brief list using the data dictionary.

In [ ]:
nfl_data_dictionary = pd.read_csv(os.path.join("NFL_play_by_play_data_dictonary.csv"), index_col = "Field")
nfl_data_dictionary.loc[["play_id", "game_id", "home_team", "away_team", "posteam",
                         "defteam", "yardline_100", "down", "ydstogo",
                        "touchdown", "play_type"]]

Use the data dictionary to look up the "game_seconds_remaining" column's definition.


<details>
nfl_data_dictionary.loc[["game_seconds_remaining"]]
</details>

Let's do a little data exploration, then start to see what we can learn about the relationships between the various features of this data set.

Create a box plot of the "down" (the offensive team must get 10 yards of field game before running out of 4 downs) and "yrdstogo" (the amount of distance to get a new set of 4 downs).


<details>
sb.boxplot(data = nfl, x = "down", y = "ydstogo")
</details>

Compute the conditional distribution of getting a touchdown ("touchdown" column) given the current "down".

In [ ]:
nfl.groupby("down")["touchdown"].value_counts(normalize = True).unstack()

<details>

Method 1:
    
```
down_touchdown_joint = nfl[["down", "touchdown"]].value_counts(normalize = True)
down_marg = nfl["down"].value_counts(normalize = True)
## conditional distribuiton of a touchdown given each down
down_touchdown_joint.div(down_marg, 0).unstack()
```

</details>

<details>
    
Method 2:
    
```
nfl.groupby("down")["touchdown"].value_counts(normalize = True).unstack()
```

</details>

## Linear Regression

Now that we have investigated the data a little, let's look at modeling **conditional means** using **linear regression**.

Recall, multiple linear regression extends our earlier use of simple linear regression by imposing the following structure:

$$E(Y \mid X_1 = x_1, X_2 = x_2, \ldots) = a + b_1 x_1 + b_2 x_2 + \ldots$$

We'll start by asking what is a typical amount of yards gained under different possible stragies.

### Yards Gained

The "yards_gained" field contains how many yards the offensive team gained (if any) on each play. Create a box plot of "yards_gained" (y-axis) broken out by "play_type" (the `showfliers = False` option is useful here).


<details>
sb.boxplot(data = nfl, x = "play_type", y = "yards_gained", showfliers = False)
</details>

As you saw, for plays other than passing and running, the yards gained was typically negligible. Let's focus in on just passing and running plays to ask the question which of these gains more yards.

In [ ]:
pass_run = nfl.loc[nfl["play_type"].isin(["run", "pass"])].copy()
pass_run["pass"] = (pass_run["play_type"] == "pass") + 0 # Indicator of passing play. Force it to be numeric

Let's start by simplying asking the question, what is the average number of yards gained for passing plays and running plays. Using `groupby` to answer this question.

In [ ]:
pass_run.groupby("pass")["yards_gained"].mean()

Now let's see that we can get the same answer using linear regression. We'll use the `sm.OLS` function  to get a linear regression of `yards_gained` on `pass`. Print out the coeffients. What do you notice about the intercpet (`const`)?


In [ ]:
tmp = pass_run[["yards_gained", "pass"]].dropna()
model = sm.OLS(tmp["yards_gained"], sm.add_constant(pass_run["pass"]))
bs = model.fit().params
bs

Now using the `bs` vector get the conditional mean of yards gained for a pass (when the variable is set to 1).


<details>
bs[0] + bs[1] * 1.0
</details>

We could group by multiple factors to get conditional means for different combinations of variables. Group by `["pass", "shotgun", "qb_dropback"]` (the second two indicate how the offense set up and how the quarterback moved after the play started) and compute the conditional mean of "yards_gained" for each combination.

Let's begin by reviewing the descriptions of these three variables.

In [ ]:
nfl_data_dictionary.loc[["pass", "shotgun", "qb_dropback"]]

<details>
pass_run.groupby(["pass", "shotgun", "qb_dropback"])["yards_gained"].mean()
</details>

This isn't too hard to read, but it starts to get tedious as we add more combinations. Additionally, dealing with continuous variable remains difficult.

Create a scatter plot of "yards_gained" (y-axis) and "game_seconds_remaining" (x-axis) using the `pass_run` data set.


<details>
sb.scatterplot(data = pass_run, x = "game_seconds_remaining", y = "yards_gained")
</details>

At first glance, this might not seem like much, but we're only looking two variables. By taking into account some of these other variables, we might learn more about how teams react as the clock clicks down. Create a regression "yards_gained" on "shotgun", "game_seconds_remaining", "pass", and "qb_dropback".

In [ ]:
X = pass_run[["shotgun", "game_seconds_remaining", "pass", "qb_dropback"]]

# use X with sm.OLS

<details>
model = sm.OLS(tmp["yards_gained"], sm.add_constant(X))

bs = model.fit().params

bs
</details>

Take a moment to answer the following questions.

On average, do teams gain more yards when:
* They use a shotgun formation
* Have the QB drop back
* The play is later in the game
* Choose to pass?


It is tempting to look at the coefficients in the previous regression and think of the **magnitude** indicating how important each variable is. But remember, they have different scales!

In the simple linear regression case ($E(Y \mid X = x) = a + bx)$), we found that $b = r_{xy} S_y / S_x$. While the expressions are a little different here, find the standard deviations of each of the variables in `X`.


<details>
X.std()
</details>

One way to understand the contributions of each variable is to replace the measurements with their Z-scores.

Take a moment to think about why this will make the measurments more comparable.

Produce the Z scores for `X` (call it `Z`). What are the standard deviations of this table?


<details>
Z = (X - X.mean()) / X.std()

Z.std()
</details>

Repeat the linear regression of "yards_gained" this time using `Z` (and `add_constant`). Call it `modelZ`. Print out the parameter values. Now that these are on the same scale, for a one standard deviation change in which of the predictors do we see the largest change in the outcome?


<details>
modelZ = sm.OLS(tmp["yards_gained"], sm.add_constant(Z))

bsZ = modelZ.fit().params

bsZ

</details>

An even better method asks the question of what is a plausible range of values for the (standardized) coefficients. Use the `.fit().conf_int()` to get 95% confidence intervals. What do you conclude about each of these variables and the relationship with yards gained, holding the others constant?



<details>
modelZ.fit().conf_int()
</details>

### Probability of a Touch Down

A consistent result in this course has been that **probabilities of events** and **means of binary (0-1) outcomes** (corresponding to those events) are actually the same thing. For binary Y,

$$P(Y = 1) = E(Y)$$

This also extends to **conditional probabilities**. So $P(Y = 1 \mid X_1 = x_1, \ldots) = E(Y \mid X_1 = x_1, \ldots)$.

Let's use `pass_run` and focus on the probability of making a touchdown. To start, let's look at the conditional probability of making a touch given the `yardline_100` variable (how far from scoring is the offensive team). Use `groupby` to compute the conditional mean of getting a touchdown for each value of `yardline_100`. Plot these results using `sb.lineplot`.


<details>
touchdown_by_yard = pass_run.groupby('yardline_100')["touchdown"].mean()

sb.lineplot(data = touchdown_by_yard) # will use the index for X and the values for Y
</details>

We notice that the probability of getting a touchdown really falls off outside of 20 yards. So let's focus on those plays within 20 yards of the end zone.

In [ ]:
within_20 = pass_run.loc[pass_run["yardline_100"] <= 20]
within_20["touchdown"].mean()

For plays within this region, what is the conditional probability of getting a touchdown given the order of down?


<details>
within_20.groupby("down")["touchdown"].mean()
</details>

If we further add pass or run as a variable (`play_type`), compute conditional probability of getting a touchdown.


<details>
within_20.groupby(["play_type", "down"])["touchdown"].mean()
</details>

As before, this starts to get tedious with more predictors and not even possible with quantitative predictors. So let's use linear regression to proceed. Fit a model on this `tmp` data set of touchdown regressed on `pass` and `down`. Print out the parameter values to see how the probability changes as a linear function of the predictors.

In [ ]:
tmp = pass_run[["touchdown", "pass", "down", "yardline_100"]].dropna()

<details>
td_mod = sm.OLS(tmp["touchdown"], sm.add_constant(tmp[["pass", "down"]]))
td_mod.fit().params
</details>

In the previous model we assume that a one unit change of "down" leads to a constant change in the conditional probability of getting a touchdown. But this may not be the case. What if each down contributes a differernt amount to the conditional probability of a touch down, even attending to whether it is a pass or run. Let's instead use the indicators of each order of downs as the predictors.

In [ ]:
tmp2 = tmp.join(pd.get_dummies(data = tmp["down"], prefix = "down")).dropna()
tmp2

In [ ]:
## notice no use of add_constant here, if we also had an intercept, we would have to discard one of the downs.
td_mod2 = sm.OLS(tmp2["touchdown"], tmp2[["pass", "down_1.0", "down_2.0", "down_3.0", "down_4.0"]].astype(float))
td_mod2.fit().params

One read of the above would be that teams should just wait for 4th downand then go for it, since the probability of getting a touch down in highest. But remember, that we are just looking at pass or run plays. Teams have the option of punting (kicking the ball away) or going for a field goal (3 points instead of 5 points). Could there be another reason why 4th downs look good when only restricted to run or pass plays?

Create a box plot with "yardline_100"on the y-axis and "down" on the x-axis.


<details>
sb.boxplot(data = pass_run, x = "down", y = "yardline_100")
</details>

Let's put it all together:

In [ ]:
td_mod3 = sm.OLS(tmp2["touchdown"], tmp2[["pass", "down_1.0", "down_2.0", "down_3.0", "down_4.0", "yardline_100"]].astype(float))
td_mod3.fit().params

Accounting for where we are the field, what do we see?


Finally for this section, let's compute the conditional probability that a team gets a touchdown on 1st down when on the 20th yard line using a running play.

<details>
bs = td_mod3.fit().params
bs["down_1.0"] + bs["yardline_100"] * 20
</details>

### Logistic Regression

In the previous investigation, we did not find any probabilities outside of the (0,1) range, but with linear regression, there is no reason this is excluded.

An alternative is **logistic regression** that passes $v = a + b_1 x_1 + ...$ through the function:

$$\frac{e^v}{1 + e^v}$$

where $e \approx 2.71$

In [ ]:
td_mod4 = slm.LogisticRegression(penalty = None,
                                 fit_intercept = False).fit(
    tmp2[["pass", "down_1.0", "down_2.0", "down_3.0", "down_4.0", "yardline_100"]],
    tmp["touchdown"])

td_mod4.coef_

These coefficients are in order of passing, the 4 downs, and distance from the end zone. Unlike linear regression, we can't interpret this result as a "one unit change leads to $b$ units change in the conditional probablity", but we can look at the signs and magnitudes to get an idea of the conditional probability changes.

We can see that touchdowns are *more likely* on pass and *less unlikely* as the down number increases. Also the farther from the end zone, touchdowns are increasing less likely.

We can compute estimated probabilties and compare them. For example, on 4th down, on the 10 yard line, assuming we don't want to punt or take a 3pt attempt, should a team run or pass to get a touchdown?

We'll use the `predict_proba` to get the predicted probabilities. The looks little tricky, but we're just creating a table with one row (the lists `[[...]]`) with values for whether teams passed (first postition) or not on 4th down (4th position) on the 10 yard line (5th position). The rest is for formatting.

In [ ]:
fourth_10_run = td_mod4.predict_proba([[0, 0, 0, 0, 1, 10]])[0][0]
fourth_10_pass = td_mod4.predict_proba([[1, 0, 0, 0, 1, 10]])[0][0]
fourth_10_run - fourth_10_pass

Here we see that there is an 8% higher likelihood that running gains a touchdown than a pass.

Let's build another logistic regression model by focusing attention on 4th down. Do or die time. The offensive team can either "go for it" by passing or running, try for lesser points with a field goal, or give the ball pack to the defense using a punt (in some cases, a few other options).

In [ ]:
fourth_down = nfl.loc[nfl["down"] == 4].copy() # the use of .copy() avoids some warning messages later

Compute the contional probability of each possible "play_type" using `value_counts`.


<details>
fourth_down["play_type"].value_counts(normalize = True)
</details>

Create a new column in the `fourth_down` table called `go_for_it` that is True when team decided to either pass or run, and False otherwise (recall the use of the `.isin(COLLECTION)` method). Show the probability of "going for it" on 4th down.


<details>

```
fourth_down["go_for_it"] = fourth_down["play_type"].isin(["pass", "run"])

fourth_down["go_for_it"].mean()
```

</details>

Create a logistic regression model that models the conditional probability of "going for it" on 4th down conditional on an intercept and the `game_seconds_remaining` column. Using the `.intercept_` and `.coef_` attributes, are teams more or less likely to go for it in the late game (when `game_seconds_remaing` is small)?



<details>
    
```
go_for_it_mod = slm.LogisticRegression(penalty = None,
                                       fit_intercept = True).fit(fourth_down[["game_seconds_remaining"]],
                                                                 fourth_down["go_for_it"])
(go_for_it_mod.intercept_, go_for_it_mod.coef_)
```

Negative coefficient means *less likely* to go for it in the early game, more likely in the late game.
</details>

## Optional: Decision Trees

After introducing linear regression as an extension of correlation, we returned to conditional means using nearest neighbor and lowess to **average over near-by points** when computing conditional means. A closely related idea, divides up the predictors over a series of steps to form a flow chart. We call these models, "decision trees."

In this section, we will mostly run and look at code, but feel free to modify these segments to ask additional questions.

To prepare for the decision tree, we do a little pre-processing:

In [ ]:
predictors = ["yardline_100", "ydstogo",
              "game_seconds_remaining", "score_differential"]

allvars = predictors.copy()
allvars.append("play_type")


fourth_down_tbl = fourth_down.get(allvars).copy()

fourth_down_tbl["play_type"] = fourth_down_tbl["play_type"].astype("category")

Now that we have the data formated the way we need it, we create a decision tree and then print it out.

In [ ]:
fourth_down_tree = tr.DecisionTreeClassifier(max_depth = 3).fit(fourth_down_tbl.get(predictors), fourth_down_tbl["play_type"].cat.codes)

In [ ]:
print(tr.export_text(fourth_down_tree, feature_names = predictors))

In [ ]:
pd.DataFrame({'code': fourth_down_tbl["play_type"].cat.codes, 'play_type': fourth_down_tbl["play_type"]}).drop_duplicates()

In [ ]:
nfl_data_dictionary.loc[["score_differential"]] # score_differential description

Using the above output, what choice would typically be made if a team is on the 30 yard line, there is more than 1.5 yards to go to get a first down, and the defensive side is leading by 12 points?
